In [1]:
import zlib
import gzip
import psycopg2
import pandas as pd

## 1. Connect with mimic database

In [2]:
conn = psycopg2.connect(host='madigun.asuscomm.com', port='5432', database='mimic', user='grace', password="grace0410")
cur = conn.cursor()

## 2. Fetch required data

In [70]:
df = pd.read_sql('''SELECT
                              DISTINCT (icd9_code)
                            FROM (SELECT
                                    subject_id,
                                    hadm_id
                                  FROM admissions a
                                  WHERE admission_type IN ('EMERGENCY', 'URGENT')) e
                              LEFT JOIN patients p
                                ON e.subject_id = p.subject_id
                              INNER JOIN (SELECT *
                                          FROM diagnoses_icd
                                          WHERE seq_num = 1
                                                AND icd9_code IS NOT NULL) AS d
                                ON e.hadm_id = d.hadm_id;''', conn)

In [71]:
df.head()

,icd9_code
0,86802
1,8742
2,2880
3,4232
4,3202


According to mimic, "the code field for the ICD-9-CM Principal and Other Diagnosis Codes is six characters in length, with the decimal point implied between the third and fourth digit for all diagnosis codes other than the V codes. The decimal is implied for V codes between the second and third digit".

- XXXXXX -> XXX.XXX
- VXXXXX -> VXX.XXX

In [106]:
def reformat_icd9(code):
    if code.find('V') <0 :
        return '.'.join([code[:3] , code[3:]])
    else:
        return '.'.join([code[:3] , code[3:]])

In [108]:
df.icd9_code = df.icd9_code.apply(lambda x: reformat_icd9(x) )

In [109]:
df.shape

(2532, 1)

## 3. Get embeddings

In [118]:
DIMENSION = 300

with gzip.open('./data/claims_codes_hs_300.txt.gz', 'rb') as f:
    file_content = f.read()

In [119]:
file_content = file_content.split(b'\n')
file_content = file_content[2:]

In [121]:
file_content  = list(filter(lambda x: x[0].find(b'IDX')>=0, map(lambda x: x.split(b' '), file_content)))

file_content = { i[0].decode('ASCII') : i[1:DIMENSION + 1] for i in file_content}

In [116]:
# file_content.keys()

In [124]:
for ind, row in df.iterrows():
    code = row['icd9_code']
#     print(code)
#     file_content['IDX_'+code]

868.02
874.2
288.0
423.2
320.2
560.1
803.75
967.9
980.0
821.39
786.05
807.02
112.4
189.3
441.6
225.2
141.0
802.8
530.3
821.31
575.0
780.39
532.61
924.11
831.01
424.1
863.83
289.89
864.13
066.41
872.00
996.73
425.11
153.9
801.01
558.1
147.8
999.5
070.31
998.13
639.0
574.90
711.01
149.0
238.0
297.8
620.1
070.41
728.88
300.00
440.20
287.30
585.9
802.28
464.30
202.70
278.8
952.04
779.3
288.02
395.2
724.02
305.00
275.41
777.8
155.0
802.7
996.86
669.34
203.10
790.92
852.05
191.0
571.42
277.87
280.0
333.0
996.79
997.01
433.21
575.5
307.81
398.91
443.24
573.8
574.40
867.7
806.25
236.0
878.0
861.13
710.1
707.0
053.0
031.0
865.14
150.8
345.11
394.9
999.89
574.00
924.10
512.89
338.3
540.1
804.16
854.04
456.0
935.1
885.0
250.33
438.11
853.06
038.44
900.82
801.74
765.27
251.2
393.
807.09
812.50
722.91
614.4
921.9
211.3
543.9
551.20
770.8
865.03
512.1
282.64
574.30
665.11
852.41
861.32
611.1
453.86
864.03
345.80
853.01
242.90
482.84
015.05
996.63
996.85
079.89
410.71
737.30
150.5
228.04
592.0
952.8


In [69]:
len(df.icd9_code.unique())

2532